In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

from glob import glob
from tqdm import tqdm

from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB3, EfficientNetB4, EfficientNetB6, ResNet50V2, DenseNet121, DenseNet169

In [ ]:
import random
import numpy as np
import tensorflow as tf

random.seed(4)
np.random.seed(4)
tf.random.set_seed(4)

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
CurrentDirectory = "/content/drive/MyDrive/efficientnet/전인서/"

train_directory = CurrentDirectory + 'train/'
test_directory  = CurrentDirectory + 'test/'
model_directory = CurrentDirectory + 'model/'
tensorboard_directory = CurrentDirectory + 'Tensorboard'

In [ ]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 32

# Training 수
epochs = 50

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001 
Decay = 1e-6

img_width = 384
img_height = 384

In [ ]:
# Online-augmentation 적용 Generator
# 1. 이미지를 전부다 불러서 램 (메모리)에 올릴 수 없기 때문
# 2. 이미지는 Augmentation을 해주는게 좋아서

DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=70,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # featurewise_center=True,
    # featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.20) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

In [ ]:
# Generator의 instance 생성 (Train)
TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "training")

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode="categorical",
    shuffle= False,
    subset = "validation")

# Generator의 instance 생성 (Test)
#TEST_GENERATOR = DATAGEN_TEST.flow_from_directory(
#    test_directory,
#    target_size = (img_width, img_height),
#    batch_size = batch_size,
#    shuffle = False,
#    class_mode='categorical')

Found 15001 images belonging to 88 classes.
Found 3707 images belonging to 88 classes.


In [ ]:
import os
import time
import shutil

train_df = pd.read_csv("/content/drive/MyDrive/efficientnet/전인서/train_df.csv")
label_list = train_df["label"].unique().tolist()
label_list = sorted(label_list)

# 폴더 안 파일 개수를 가져오는 함수
def get_num_of_files(directory):
  dirListing = os.listdir(directory)
  return len(dirListing)

# 이미지 개수 안 늘려진 폴더 리스트
to_fix_list = []
for label in label_list:
  num_files = get_num_of_files(f'/content/drive/MyDrive/efficientnet/전인서/train/{label}/')
  if num_files < 200:
    to_fix_list.append(label)

# 추가 증강
for i in range(len(to_fix_list)):
  current_dir = f'/content/drive/MyDrive/efficientnet/전인서/train/{to_fix_list[i]}'
  
  original_file_list = []
  for file in os.listdir(current_dir):
    if len(file)== 9:
      original_file_list.append(file)
  
  whole_file_list = os.listdir(current_dir)

  while len(whole_file_list) < 200:
    for file in original_file_list:
      t = float(time.time())
      shutil.copyfile(f'{current_dir}/{file}', f'{current_dir}/{t}-{file}')
    whole_file_list = os.listdir(current_dir)
  
  print(f'{i} {to_fix_list[i]} complete')


# Generator의 instance 생성 (Test)
TEST_GENERATOR = DATAGEN_TEST.flow_from_directory(
    test_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical')

In [ ]:
len(pd.Series( VALID_GENERATOR.labels ).value_counts())
# len(pd.Series( TRAIN_GENERATOR.labels ).value_counts())

88

In [ ]:
Model_1 = tf.keras.applications.DenseNet121(
    include_top=None,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_width,img_height,3),
    pooling=None)


x = GlobalAveragePooling2D()(Model_1.output)
predictions = Dense(88, activation='softmax')(x)

DenseNet = Model(inputs=Model_1.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
DenseNet.compile(optimizer=
         SGD(learning_rate=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc']) 


29097984/29084464 [==============================] - 0s 0us/step


In [ ]:
# Call-back 함수

# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath=model_directory+'DenseNet-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# 학습과정 진행사항 확인
TB = TensorBoard(log_dir=tensorboard_directory, write_graph=True, write_images=True)

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3, verbose=1, min_lr=1e-7)

CALLBACK = [CP, TB, LR]

In [ ]:
DenseNet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 390, 390, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 192, 192, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [ ]:
DenseNet.load_weights(CurrentDirectory + "model/DenseNet-019-0.0051-0.9981.hdf5")

In [ ]:
########## Training Start
DenseNet.fit_generator(
        TRAIN_GENERATOR,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        
        epochs=25,
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)
###########

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/25
469/469 [==============================] - ETA: 0s - loss: 0.0072 - acc: 0.9988
Epoch 1: val_loss improved from inf to 0.00832, saving model to /content/drive/MyDrive/efficientnet/전인서/model/DenseNet-001-0.0083-0.9973.hdf5
469/469 [==============================] - 4356s 9s/step - loss: 0.0072 - acc: 0.9988 - val_loss: 0.0083 - val_acc: 0.9973 - lr: 0.0010
Epoch 2/25
469/469 [==============================] - ETA: 0s - loss: 0.0085 - acc: 0.9981
Epoch 2: val_loss did not improve from 0.00832
469/469 [==============================] - 1377s 3s/step - loss: 0.0085 - acc: 0.9981 - val_loss: 0.0150 - val_acc: 0.9957 - lr: 0.0010
Epoch 3/25
469/469 [==============================] - ETA: 0s - loss: 0.0066 - acc: 0.9987
Epoch 3: val_loss did not improve from 0.00832
469/469 [==============================] - 1449s 3s/step - loss: 0.0066 - acc: 0.9987 - val_loss: 0.0141 - val_acc: 0.9965 - lr: 0.0010
Epoch 4/25
469/469 [==============================] - ETA: 0s - loss: 0.0074 - acc: